In [1]:
import torch
import os, sys
sys.path.insert(0, '../src')
import wandb
import pickle as pkl
import numpy as np
from scipy.stats import gaussian_kde
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from data_generator import SpatialDataset, train_val_test_split, train_val_test_split_gps
from models import LinearSCI, NonlinearSCI
from trainers import Trainer, GPSModelTrainer
from gps import GeneralizedPropensityScoreModel

# Experimental tracking
# wandb.login()

np.random.seed(2020)
torch.manual_seed(2020)
torch.cuda.manual_seed(2020)
torch.cuda.manual_seed_all(2020)
torch.backends.cudnn.deterministic = True

# Load data

In [2]:
with open('../data/synthetic_data.pkl', 'rb') as fp:
    data = pkl.load(fp)
neighborhood_size, num_bins = data['neighborhood_size'], 50
T = np.concatenate(
    [data['T_bar'][:,:neighborhood_size], 
     data['T'], 
     data['T_bar'][:,neighborhood_size:]], axis=1)
X, Y, s = data['X'], data['Y'], data['s']
de, ie, te = data['de'], data['ie'], data['te']
Y_direct, Y_indirect, Y_total = data['Y_direct'], data['Y_indirect'], data['Y_total']
T_min, T_max = np.min(data['T']), np.max(data['T'])

train_dataset, val_dataset, test_dataset = train_val_test_split(
    t=[T], x=X, s=s, y=Y, train_size=0.6, val_size=0.2, test_size=0.2, 
    shuffle=True, random_state=2020
)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=50, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)
dataloaders = {'train': train_loader, 'val': val_loader, 'test': test_loader}

train_dataset_gps, val_dataset_gps, _ = train_val_test_split_gps(
    t=data['T'], x=X, s=s, train_size=0.8, val_size=0.2, test_size=0.0, 
    shuffle=True, random_state=2020
)
train_loader_gps = DataLoader(train_dataset_gps, batch_size=50, shuffle=True)
val_loader_gps = DataLoader(val_dataset_gps, batch_size=50, shuffle=False)
dataloaders_gps = {'train': train_loader_gps, 'val': val_loader_gps, 'test': None}

# Generalized Propensity Model and Stabilized Weights

In [ ]:
# GPS model
gps_model = GeneralizedPropensityScoreModel(
    input_dim=X.shape[1]+s.shape[1],
    num_hidden_layers=1,
    hidden_dims=16
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optim = "sgd"
optim_params = {
    'lr': 1e-3, 
    'momentum': 0.99
}
epochs, patience = 1000, 50
trainer = GPSModelTrainer(
    model=gps_model, 
    data_generators=dataloaders_gps, 
    optim=optim, 
    optim_params=optim_params, 
    device=device,
    epochs=epochs,
    patience=patience
)
trainer.train()

# Stabilized weights model
sw_model = gaussian_kde(data['T'].squeeze())

# Method 1: Linear model without U

In [ ]:
num_iterations = 5
de_error, ie_error, te_error = [], [], []

for _ in tqdm(range(num_iterations), position=0, leave=True):
    model = LinearSCI(
        num_interventions=1, 
        window_size=neighborhood_size*2+1, 
        confounder_dim=X.shape[1]
    )
    
    # Training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optim = "sgd"
    optim_params = {
        'lr': 1e-3, 
        'momentum': 0.99, 
        'weight_decay': 0.01
    }
    epochs, patience = 1000, 50
    trainer = Trainer(
        model=model, 
        data_generators=dataloaders, 
        optim=optim, 
        optim_params=optim_params, 
        window_size=neighborhood_size*2+1, 
        t_idx=0, 
        gps_model=gps_model, 
        sw_model=sw_model, 
        device=device,
        epochs=epochs,
        patience=patience
    )
    trainer.train()

    # Evaluation
    Y_direct_pred = trainer.predict(mode='direct', t_min=T_min, t_max=T_max, num_bins=num_bins, weighting='snipw')
    Y_indirect_pred = trainer.predict(mode='indirect', t_min=T_min, t_max=T_max, num_bins=num_bins, weighting='snipw')
    Y_total_pred = trainer.predict(mode='total', t_min=T_min, t_max=T_max, num_bins=num_bins, weighting='snipw')
    de_pred = np.mean(np.mean(Y_direct_pred, axis=1), axis=0)
    ie_pred = np.mean(np.mean(Y_indirect_pred, axis=1), axis=0)
    te_pred = np.mean(np.mean(Y_total_pred, axis=1), axis=0)
    de_error.append(np.abs(de_pred - de))
    ie_error.append(np.abs(ie_pred - ie))
    te_error.append(np.abs(te_pred - te))

In [5]:
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average direct effect = {np.mean(de_error):.5f} +/- {np.std(de_error):.5f}")
print(f"Average indirect effect = {np.mean(ie_error):.5f} +/- {np.std(ie_error):.5f}")
print(f"Average total effect = {np.mean(te_error):.5f} +/- {np.std(te_error):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average direct effect = 0.70925 +/- 0.02269
Average indirect effect = 0.80613 +/- 0.07961
Average total effect = 0.78086 +/- 0.07064


# Model 2: Linear model with U

In [ ]:
num_iterations = 5
de_error, ie_error, te_error = [], [], []

for _ in tqdm(range(num_iterations), position=0, leave=True):
    model = LinearSCI(
        num_interventions=1, 
        window_size=neighborhood_size*2+1, 
        confounder_dim=X.shape[1], 
        unobserved_confounder=True, 
        kernel_param_vals=[1.,0.5,0.5]
    )
    
    # Training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optim = "sgd"
    optim_params = {
        'lr': 1e-3, 
        'momentum': 0.99, 
        'weight_decay': 0.01
    }
    epochs, patience = 1000, 50
    trainer = Trainer(
        model=model, 
        data_generators=dataloaders, 
        optim=optim, 
        optim_params=optim_params, 
        window_size=neighborhood_size*2+1, 
        t_idx=0, 
        gps_model=gps_model, 
        sw_model=sw_model, 
        device=device,
        epochs=epochs,
        patience=patience
    )
    trainer.train()

    # Evaluation
    Y_direct_pred = trainer.predict(mode='direct', t_min=T_min, t_max=T_max, num_bins=num_bins, weighting='snipw')
    Y_indirect_pred = trainer.predict(mode='indirect', t_min=T_min, t_max=T_max, num_bins=num_bins, weighting='snipw')
    Y_total_pred = trainer.predict(mode='total', t_min=T_min, t_max=T_max, num_bins=num_bins, weighting='snipw')
    de_pred = np.mean(np.mean(Y_direct_pred, axis=1), axis=0)
    ie_pred = np.mean(np.mean(Y_indirect_pred, axis=1), axis=0)
    te_pred = np.mean(np.mean(Y_total_pred, axis=1), axis=0)
    de_error.append(np.abs(de_pred - de))
    ie_error.append(np.abs(ie_pred - ie))
    te_error.append(np.abs(te_pred - te))

In [7]:
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average direct effect = {np.mean(de_error):.5f} +/- {np.std(de_error):.5f}")
print(f"Average indirect effect = {np.mean(ie_error):.5f} +/- {np.std(ie_error):.5f}")
print(f"Average total effect = {np.mean(te_error):.5f} +/- {np.std(te_error):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average direct effect = 0.10790 +/- 0.01160
Average indirect effect = 0.12261 +/- 0.00959
Average total effect = 0.11484 +/- 0.01790


# Model 3: Nonlinear model without U (f: MLP, g:MLP)

In [ ]:
num_iterations = 5
de_error, ie_error, te_error = [], [], []

for _ in tqdm(range(num_iterations), position=0, leave=True):
    model = NonlinearSCI(
        num_interventions=1, 
        window_size=neighborhood_size*2+1, 
        confounder_dim=X.shape[1], 
        f_network_type="mlp", 
        f_hidden_dims=[256,256], 
        g_network_type="mlp", 
        g_hidden_dims=[256,256]
    )
    
    # Training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optim = "sgd"
    optim_params = {
        'lr': 1e-3, 
        'momentum': 0.99, 
        'weight_decay': 0.01
    }
    epochs, patience = 1000, 50
    trainer = Trainer(
        model=model, 
        data_generators=dataloaders, 
        optim=optim, 
        optim_params=optim_params, 
        window_size=neighborhood_size*2+1, 
        t_idx=0, 
        gps_model=gps_model, 
        sw_model=sw_model, 
        device=device,
        epochs=epochs,
        patience=patience
    )
    trainer.train()

    # Evaluation
    Y_direct_pred = trainer.predict(mode='direct', t_min=T_min, t_max=T_max, num_bins=num_bins, weighting='snipw')
    Y_indirect_pred = trainer.predict(mode='indirect', t_min=T_min, t_max=T_max, num_bins=num_bins, weighting='snipw')
    Y_total_pred = trainer.predict(mode='total', t_min=T_min, t_max=T_max, num_bins=num_bins, weighting='snipw')
    de_pred = np.mean(np.mean(Y_direct_pred, axis=1), axis=0)
    ie_pred = np.mean(np.mean(Y_indirect_pred, axis=1), axis=0)
    te_pred = np.mean(np.mean(Y_total_pred, axis=1), axis=0)
    de_error.append(np.abs(de_pred - de))
    ie_error.append(np.abs(ie_pred - ie))
    te_error.append(np.abs(te_pred - te))

In [9]:
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average direct effect = {np.mean(de_error):.5f} +/- {np.std(de_error):.5f}")
print(f"Average indirect effect = {np.mean(ie_error):.5f} +/- {np.std(ie_error):.5f}")
print(f"Average total effect = {np.mean(te_error):.5f} +/- {np.std(te_error):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average direct effect = 0.07780 +/- 0.04942
Average indirect effect = 0.23065 +/- 0.05397
Average total effect = 0.34749 +/- 0.09264


# Model 4: Nonlinear model with U (f: MLP, g: MLP)

In [ ]:
num_iterations = 5
de_error, ie_error, te_error = [], [], []

for _ in tqdm(range(num_iterations), position=0, leave=True):
    model = NonlinearSCI(
        num_interventions=1, 
        window_size=neighborhood_size*2+1, 
        confounder_dim=X.shape[1], 
        f_network_type="mlp", 
        f_hidden_dims=[256,256], 
        g_network_type="mlp", 
        g_hidden_dims=[256,256], 
        unobserved_confounder=True, 
        kernel_param_vals=[1.,0.5,0.5]
    )
    
    # Training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optim = "sgd"
    optim_params = {
        'lr': 1e-3, 
        'momentum': 0.99, 
        'weight_decay': 0.01
    }
    epochs, patience = 1000, 50
    trainer = Trainer(
        model=model, 
        data_generators=dataloaders, 
        optim=optim, 
        optim_params=optim_params, 
        window_size=neighborhood_size*2+1, 
        t_idx=0, 
        gps_model=gps_model, 
        sw_model=sw_model, 
        device=device,
        epochs=epochs,
        patience=patience
    )
    trainer.train()

    # Evaluation
    Y_direct_pred = trainer.predict(mode='direct', t_min=T_min, t_max=T_max, num_bins=num_bins, weighting='snipw')
    Y_indirect_pred = trainer.predict(mode='indirect', t_min=T_min, t_max=T_max, num_bins=num_bins, weighting='snipw')
    Y_total_pred = trainer.predict(mode='total', t_min=T_min, t_max=T_max, num_bins=num_bins, weighting='snipw')
    de_pred = np.mean(np.mean(Y_direct_pred, axis=1), axis=0)
    ie_pred = np.mean(np.mean(Y_indirect_pred, axis=1), axis=0)
    te_pred = np.mean(np.mean(Y_total_pred, axis=1), axis=0)
    de_error.append(np.abs(de_pred - de))
    ie_error.append(np.abs(ie_pred - ie))
    te_error.append(np.abs(te_pred - te))

In [11]:
print("Error on prediction of average local and interference effects:")
print("--------------------------------------------------------------")
print(f"Average direct effect = {np.mean(de_error):.5f} +/- {np.std(de_error):.5f}")
print(f"Average indirect effect = {np.mean(ie_error):.5f} +/- {np.std(ie_error):.5f}")
print(f"Average total effect = {np.mean(te_error):.5f} +/- {np.std(te_error):.5f}")

Error on prediction of average local and interference effects:
--------------------------------------------------------------
Average direct effect = 0.09552 +/- 0.01616
Average indirect effect = 0.11573 +/- 0.00453
Average total effect = 0.09545 +/- 0.01625
